In [9]:
!pip install faster-whisper

Defaulting to user installation because normal site-packages is not writeable


In [10]:
!pip install opus-fast-mosestokenizer

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install opus-fast-mosestokenizer

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.2/837.2 kB 5.2 MB/s eta 0:00:0000:0100:01


To chose a better GPU 

In [1]:
import GPUtil

gpu_num = None

# Получение списка GPU
gpus = GPUtil.getGPUs()

# Фильтрация списка GPU, исключая GPU с ID 6
filtered_gpus = [gpu for gpu in gpus if gpu.id != '6']

# Если отфильтрованные GPU не найдены, выводим сообщение об этом
if not filtered_gpus:
    print("GPU, подходящих под условия, не обнаружено.")
else:
    # Находим GPU с наибольшим количеством свободной памяти среди отфильтрованных
    max_memory_gpu = max(filtered_gpus, key=lambda gpu: gpu.memoryFree)
    print(f"GPU с наибольшим количеством свободной памяти (кроме GPU 6): {max_memory_gpu.id}")
    gpu_num = max_memory_gpu.id

GPU с наибольшим количеством свободной памяти (кроме GPU 6): 1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

Realtime whisper server

In [10]:
import socket
import threading
from pydub import AudioSegment
from pydub.playback import play
import io
import pyaudio
import time
from whisper_online import *

src_lan = "ru"  # source language
tgt_lan = "ru"  # target language  -- same as source for ASR, "en" if translate task is used

host="127.0.0.1"
port=65432

asr = FasterWhisperASR("ru", "large-v2")  # loads and wraps Whisper model
# set options:
# asr.set_translate_task()  # it will translate from lan into English
# asr.use_vad()  # set using VAD 


online = OnlineASRProcessor(tgt_lan, asr)  # create processing object

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            data = conn.recv(16000)
            if not data:
                break
            else:
                data = np.frombuffer(data, dtype=np.int16)
                data = data.astype(np.float32)/(2**(2*8-1))
                online.insert_audio_chunk(data)
                o = online.process_iter()
                print(o) # do something with current partial output
            
# at the end of this audio processing
o = online.finish()
print(o)  # do something with the last output


online.init()  # refresh if you're going to re-use the object for the next audio

Connected by ('127.0.0.1', 55274)


AttributeError: 'str' object has no attribute 'sep'

Test server

In [2]:
import socket
import numpy as np

def is_silent(data, threshold=500):
    """Определение тишины в аудиопотоке."""
    amplitudes = np.frombuffer(data, dtype=np.int16)
    return np.abs(amplitudes).mean() < threshold

def start_server(host="127.0.0.1", port=65432):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")
            while True:
                data = conn.recv(2048)
                if not data:
                    break
                if is_silent(data):
                    print("Тихо...")
                else:
                    print("Есть звук!")

if __name__ == "__main__":
    start_server()


KeyboardInterrupt: 